Codierung multimedialer Daten
=============================

**Author:** Johann-Markus Batke

**Date:** <2024-05-08 Mi>



## Korrelation



Bestimmen Sie die Korrelation zwischen linkem und rechtem Signal eines Stereosignals. Verwenden Sie dazu die Programmierumgebung Ihrer Wahl.

initialisiere benötigte Module:



In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.io.wavfile as wavfile

Musik-Datei einlesen:



In [1]:
S_filename = "../Vorlesung/1Battle_Royal.wav"
[f_rate, x_data] = wavfile.read(S_filename)
x_L = np.array(x_data, dtype=float)
plt.plot(x_L)

### Wahrscheinlichkeitsdichtefunktion



Schätzung der Wahrscheinlichkeitsdichtefunktion $f_{\vec x}(x)$ per Häufigkeit



In [1]:
bins_x = np.arange(np.min(np.min(x_data)), np.max(np.max(x_data))+1)
f_1_x, bin_edge = np.histogram(x_data[:,0], bins_x, density=True)
f_2_x, bin_edge = np.histogram(x_data[:,1], bins_x, density=True)
plt.plot(bins_x[1:bins_x.size], f_1_x*100)
plt.plot(bins_x[1:bins_x.size], f_2_x*100)
plt.xlabel("$x$")
plt.ylabel("rel. Haeufigkeit in %")

Text(0, 0.5, 'rel. Haeufigkeit in %')

### Wahrscheinlichkeitsverteilung



Wahrscheinlichkeitsverteilung $F_{\vec x}(x)$



In [1]:
F_1 = np.cumsum(f_1_x)
F_2 = np.cumsum(f_2_x)
plt.plot(bins_x[1:bins_x.size], F_1)
plt.plot(bins_x[1:bins_x.size], F_2)
plt.xlabel("$x$")
plt.ylabel("$F_{x}(x)$")

Text(0, 0.5, '$F_{x}(x)$')

### Erwartungswert



Erwartungswert $E\left\{ \vec x \right\} = \int\limits_{-\infty}^{\infty} x \, f_{x}(x) \mathrm dx$



In [1]:
def erwartungswert(x, f_x):
    dx = np.diff(x[0:2])
    return (f_x.reshape(1, f_x.size) @ x.reshape(x.size, 1)) * dx

### Definition k-tes Moment



\begin{equation}
m_{\vec x}^{(k)} = \mathrm E \left\{ \vec x^{k}(\eta) \right\}
\end{equation}



#### Linearer Mittelwert



$m_{\vec x}^{(1)} = E\left\{ \vec x \right\}$



In [1]:
x = bins_x[0:bins_x.size-1]
mu_1 = erwartungswert(x, f_1_x)
print(mu_1)
mu_2 = erwartungswert(x, f_2_x)
print(mu_2)

[[-69.6974172]]
[[-44.52540321]]

#### quadratischer Erwartungswert



$m_{\vec x}^{(2)} = E\left\{ \vec x^2 \right\} = \int\limits_{-\infty}^{\infty} x^{2} \, f_{x}(x) \mathrm dx$



In [1]:
x = bins_x[0:bins_x.size-1]
mu_1 = erwartungswert(x**2, f_1_x)
print(mu_1)
mu_2 = erwartungswert(x**2, f_2_x)
print(mu_2)

[[-2.15623768e+11]]
[[-2.46215762e+11]]

### Definition k-tes zentrales Moment



\begin{equation}
\label{eq:kmoment}
\mu_{\vec x}^{(k)} = \mathrm E \left\{ \left(\vec x - m_{\vec x}^{(1)}  \right)^{k} \right\}
\end{equation}



#### Varianz



$\mu_{\vec x}^{(2)} = \sigma_{\vec x}^{2} = E\left\{ \vec (x-m_{\vec x}^{(1)})^2 \right\}$

Es gilt: $\sigma_{\vec x}^{2} = m_{\vec x}^{(2)} - \left( m_{\vec x}^{(1)}\right)^{2}$



### Zeitmittelwert



$\overline{x} = \sum \frac{x[n]}{N}$



In [1]:
mu_x = sum(x_L, 0)/x_L.shape[0]
print(mu_x)

[-69.6974172  -44.52539945]

sind nahe Null.



### quadratischer Zeitmittelwert



$\overline{x^{2}} = \sum \frac{x[n]^{2}}{N}$



In [1]:
sigma_x = sum(x_L**2, 0)/x_L.shape[0]
print(sigma_x)

[6113690.99554119 6981081.59907139]

### Mittelwerte zwischen den Kanälen



In [1]:
x_mu_L = np.sum(x_L, 1)/x_L.shape[1]
plt.plot(x_mu_L)
plt.xlabel("$n$")

Text(0.5, 0, '$n$')

### Korrelation zwischen den Kanälen



Ein Stereosignal mit den Abtastwerten $x_{L}$ und $x_{R}$ bildet das Produkt



In [1]:
x_si_L =x_L[:,0] * x_L[:,1]
plt.plot(x_si_L)
plt.xlabel("$n$")

Text(0.5, 0, '$n$')

Das Ergebnis ist überwiegend größer Null, da die beide Signale
meistens das gleiche Vorzeichen haben.



### Korrelation im Signal



Das Produkt zweier aufeinanderfolgender Abtastwerte ist:



In [1]:
x_korr_Lm1 = x_L[1:-1] * x_L[0:x_L.shape[0]-2]/x_L.shape[0]
plt.plot(x_korr_Lm1)

| <matplotlib.lines.Line2D | at | 0x7fe70c1fe0d0> | <matplotlib.lines.Line2D | at | 0x7fe70b8f30d0> |

Das Produkt ist überwiegend positiv, da die benachbarten Abtastwerte das gleiche Vorzeichen haben.



### Korrelationskoeffizient



\begin{equation}
\label{eq:korrkoeff}
\rho_{\vec x_{1}\vec x_{2}} = \frac{\mu_{\vec x_{1}\vec x_{2}}^{(1,1)}}{\sigma_{\vec x_{1}}\sigma_{\vec x_{2}}}
\end{equation}



In [1]:
x_ss_L =(x_L[:,0]**2 * x_L[:,1]**2)**0.5
isntzero = np.where(x_ss_L != 0)
rho = x_si_L[isntzero]/x_ss_L[isntzero]
plt.plot(rho[0:200])
plt.xlabel("$n$")

Text(0.5, 0, '$n$')

### Statistische Abhängigkeit



Vollziehen Sie das Beispiel aus 3.8.1 mit einem eigenen Bild in einer eigenen Implementierung nach.

